# Downstream Analysis of Single-Cell Data  -- <font color='orange'>Solutions</font>

<br><font color='red'>**Warning 1:** These materials were put together relatively quickly and have not been thoroughly double-checked and optimized yet! Any feedback that may help to improve these materials is very welcome! </font><br><br>

<font color='red'>**Warning 2:** These materials use an example image with a quite uniform population of cells (meaning the cells are in general quite similar). As a consequence, there aren't really any interesting patterns that can be uncovered by data analysis, which is a bit unfortunate for a tutorial on this topic. However, the concepts can still be explained - it's just that the outcome won't be too exciting. We are planning to switch to more interesting example images in the future.</font>

## Introductory Notes


### About this Tutorial

This tutorial provides an introductory overview of how to approach single-cell analysis following image segmentation.  

For people with limited experience in data analysis, this tutorial is intended as an inspiration and incentive to think about possible advanced analyses downstream of segmentation. Solving the exercises without help may be difficult, so it may be a good idea to have a look at the solutions to get some idea of how the problems should be approached. However, once the principles are understood, it is an important part of the learning experience to build your own implementation and to play around with different possibilities.

People more experienced in data science can use this tutorial as a starting point for exploring the data analysis packages available in Python. It also illustrates that Python readily allows the construction of complete and consistent analysis pipelines, from image preprocessing to feature extraction to clustering. 


### Concepts Discussed in this Tutorial

- Feature extraction
    - Specific features of interest
    - General image descriptors


- Feature space standardization


- Dimensionality reduction
    - PCA
    - tSNE


- Clustering by k-means (including Elbow plots)


- Graph/network-based data representation and visualization



### Data Analysis with Python

There are a number of data analysis, machine learning, clustering and other data analysis packages for Python. The following is a list of some commonly used packages:

- [`pandas`](http://pandas.pydata.org/)
    - Provides a dataframe object ideal for handling `sample x feature` data
    - Most of the packages mentioned below can seamlessly work with `pandas` dataframes
    - *Note:* the solutions to this tutorial currently do *not* use `pandas` *but will be updated to do so in the future!*


- [scikit-learn (`sklearn`)](http://scikit-learn.org/stable/)
    - Large data analysis and machine learning package featuring many standard and state-of-the-art algorithms


- [`scipy.cluster`](http://docs.scipy.org/doc/scipy/reference/cluster.html)
    - Scipy implementation of clustering algorithms


- [`networkx`](http://networkx.github.io/)
    - Package for graph-based analysis and visualization of data


- [`keras`](https://keras.io/)
    - Extensive state-of-the-art package for deep learning
    - Provides APIs for Theano and TensorFlow
    - *Note:* this tutorial currently does *not* cover deep learning
    

- [`pyMC`](https://pymc-devs.github.io/pymc/)
    - Package for Bayesian modeling and Markov-Chain Monte-Carlo sampling
    - *Note:* this tutorial currently does *not* cover Bayesian inference

Most of these packages use `numpy` arrays (or `pandas` dataframes) to handle data, so being familiar with `numpy` is an essential prerequisite to working with any of the above. If you have completed the main tutorial, you should not have any major problems with this.


### Setup

The following packages/modules need to be installed for this tutorial. With the exception of `tifffile`, they should all come pre-installed with the Anaconda distribution or sould be easy to install using `conda install` or `pip install`.

- **Already needed for main tutorial**
    - python 2.7 *(or python 3.x, if you don't mind adjusting the solutions)*
    - `numpy`
    - scikit-image (`skimage`)
    - `matplotlib`
    - `scipy`
    - `tifffile`


- **New for this tutorial**
    - scikit-learn  (`sklearn`)
    - `networkx`

Make sure these modules are installed before you proceed.

Note that this tutorial is based on the segmentation and extracted measures from `example_image_1.tif` from the main pipeline. The necessary files to run the solutions are included in this directory, but it might be a bit more interesting for you to use the results of your own work.

## Preparation

In [ ]:
### Importing modules

from __future__ import division    # Python 2.7 legacy (not necessary if working with python v3.x)
import numpy as np                 # Array manipulation package
import matplotlib.pyplot as plt    # Plotting package
import scipy.ndimage as ndi        # Multidimensional image operations

import sklearn as skl              # Data analysis and machine learning
import networkx as nx              # Graphs/networks

import json                        # File handling

### <font color='orange'>Exercise 0  (Solution)</font>

Reload the image, the segmentation, and the results dictionary from the main tutorial. The labeled cell edges may also be useful.

In [ ]:
### Loading image and segmentation data from the main tutorial

# Filename
filename = 'example_cells_1.tif'

# Load the image
from tifffile import imread
img = imread(filename)
img_green = img[0,:,:]
img_red   = img[1,:,:]

# Load the cell edges
edges = imread(filename[:-4]+'_edges.tif')

# Load the segmentation
seg = np.load(filename[:-4]+'_seg.npy')

# Load the results dictionary
with open(filename[:-4]+'_results.json', 'r') as infile:
    res = json.load(infile)

# Some frequently used variables
labels = np.unique(seg)[1:]  # Labels of cells in segmentation
N = len(labels)              # Number of cells in segmentation

# Report
print "Loaded file", filename
print "  Number of cells:   ", N
print "  Number of features:", len(res)-1

# SOLUTION NOTE: If you're working through these tutorials in a Jupyter Notebook, you could
#                also use the Jupyter magic %store command to share the variables between
#                your main tutorial notebook and this one

## Feature Extraction

As discussed in the main tutorial, we can measure various quantities for each cell individually (once the cells have been segmented). These quantities can be considered 'features' for the purpose of further analysis such as clustering. Besides explicitly measured specific quantities, there are also algorithms that automatically extract a whole bunch of features from an image.

All the extracted features together are called the 'feature space'. Each sample can be considered a point in feature space, which has as many dimensions as there are features. The feature space should be arranged as an array of shape `(n_samples,n_features)` or in an equivalent pandas `df`.

### <font color='orange'>Exercise 1  (Solution)</font>

You should already have a set of single-cell measurements ('features') from the main tutorials, stored in the results dict.

Try to think of a few (at least 2) additional features and extract them from the image/segmentation. Be creative and try to find features that might contain biologically interesting information!

Combine all the features in a 'feature_space' array of shape `(n_samples,n_features)`, where `n_samples` in our case is the number of cells. You could also use a pandas dataframe for this. Be sure to use a data type that works for all the different types of features you are using. If you are using numpy arrays, it makes sense to also keep a list of column labels (names of the features) and row labels (cell IDs) because the array itself is unlabeled.

- **Hint 1:** For many measures of shape and spatial distribution, it is useful to first calculate the `centroid` of the segmented object and then think of features relative to it.


- **Hint 2:** It can be advantageous to use measures that are largely independent of cell size (or normalized for cell size) to prevent cell size from dominating the different features. Of course, cell size itself should be a feature.


- **Hint 3:** Don't forget that we also identified the boundaries of each cell in the main script. Importing or reconstructing this data here may be useful for the calculation of various features.


- **Hint 4:** Make sure you visualize your data!
    - It can be very useful to have a look at what a feature looks like when mapped to the actual image (using a semi-transparent overlay, see section 'Expansion by Watershed' in the main tutorial). 
    - This may already show interesting patterns, or should at least confirm that the extracted values are consistent with the feature's rationale.
    - Also, box and scatter plots are great options for checking how the values of a feature are distributed and how features relate to each other.
    

<font color='orange'>**Solution note:**</font> The following are just suggestions/ideas! Your solution should be conceptually similar but not at all the same. This also goes for the rest of the pipeline, both because there are multiple ways of achieving the same or a similar goal, and also because your solutions here will change the outcome of the analyses further below!

In [ ]:
### Initialize feature space array and add previously extracted features

# Initialize feature space
fspace = np.zeros((N,9),dtype=np.float)

# Labels
col_labels = []

# Useful index to paste things into fspace array
col_index  = 0

# Add previously extracted measures to feature space
for measure in res.keys():
    if not measure == 'cell_id':
        col_labels.append(measure)
        fspace[:,col_index] = res[measure]
        col_index += 1

print col_labels
print fspace.mean(axis=0)

In [ ]:
### Get additional features and add them to the feature space

# Get the centroids
cen = np.zeros((N,2))
for row_index,label in enumerate(labels):
    cen[row_index,:] = ndi.measurements.center_of_mass(seg==label)


# FEATURE: Standard deviation of membrane intensity
# Depending on the channel used and the sample, a feature like this could relate 
# to cell polarity, membrane ruffling, protrusions, receptor clustering, 
# endocytosis, junctions, ...

# Iterate over cells
for row_index,label in enumerate(labels):
    
    # Get the membrane standard deviation
    fspace[row_index,col_index] = np.std(img_green[edges==label])

# Update column label
col_labels.append("green_mem_stdev")
col_index += 1


# FEATURE: Intensity asymmetry as the distance of the intensity-weighted centroid
#          from the unweighted centroid.
# This feature could be useful to look at cell polarity. 

# Iterate over cells
for row_index,label in enumerate(labels):
    
    # Intensity-weighted center of mass
    int_cen = ndi.measurements.center_of_mass(np.ma.array(img_green,mask=edges!=label))
    
    # Distance between the two centroids
    # Note: Below I use the pairwise distance function pdist from scipy's
    #       spatial package. This function is useful for a lot of things in
    #       data analsis, but to be more explicit one could also calculate the
    #       distance between two centroids using simple Pythagoras:
    #           diff = int_cen - cen[index,:]
    #           dist = np.sqrt(np.square(diff[0]) + np.square(diff[1]))
    from scipy.spatial.distance import pdist
    fspace[row_index,col_index] = pdist([int_cen,cen[row_index,:]])[0]

# Update column label
col_labels.append("green_asymmetry")
col_index += 1


# FEATURE: Roundnes of the cells as the ratio of inscribed vs circumscribed circle
# This feature could relate to the forces excerted on a cell or to polarity.
# It may also be able to detect the rounding of a cell as it enters division.

# Iterate over cells
for row_index,label in enumerate(labels):
    
    # Distance transform to get circular distance around centroid
    dtr = np.ones_like(seg)
    dtr[int(cen[row_index,0]),int(cen[row_index,1])] = 0
    dtr = ndi.distance_transform_edt(dtr)
    
    # Calculate circle radii and their ratio
    in_rad  = np.min(np.ma.array(dtr,mask=seg==label))  # Maximum inscribed circle radius 
    out_rad = np.max(np.ma.array(dtr,mask=seg!=label))  # Minimum circumscribed circle
    fspace[row_index,col_index] = in_rad / out_rad              # Ratio

# Update column label
col_labels.append("roundness")
col_index += 1

# Report
print col_labels
print fspace.mean(axis=0)

### <font color='orange'>Exercise 2  (Solution)</font>

Use a feature extraction algorithm that returns a large feature set for each cell; an image **descriptor**. The features could for example be related to shape or texture. Feel free to search around the internet for a bit to see what kind of interesting algorithms are available. The algorithm should produce a second feature space that once again has the shape `(n_samples,n_features)`.

The one used in the solutions is `skimage.feature.daisy`, an algorithm for the extraction of local image features based on a grayscale image. It should be pointed out, however, that membrane images are not what [DAISY](scikit-image.org/docs/dev/api/skimage.feature.html#skimage.feature.daisy) would typically be applied to; it is best used for images with more local patterning/texture.

In [ ]:
### Extraction of DAISY image descriptor features

# Initialize feature space
fspace2 = np.zeros((N,200))

# Get the daisy function
from skimage.feature import daisy

# Loop over cells
for index,label in enumerate(labels):
    
    # Crop image to bounding box of the cell of interest
    # Note: the DAISY image descriptors may be partially sensitive to the size
    #       of the image it is being computed from. Thus, cropping the image to
    #       bounding boxes of different sizes may not be the ideal strategy here!
    non_zero_indices = np.nonzero(seg==label)
    y_min = np.min(non_zero_indices[0])
    y_max = np.max(non_zero_indices[0])
    x_min = np.min(non_zero_indices[1])
    x_max = np.max(non_zero_indices[1])
    cell_img = img_green[y_min:y_max,x_min:x_max]
    
    # Extract DAISY features and build a feature space with a subset of them
    # Note that some of the segmentation artefacts can cause an error here;
    # these cells are marked as NaN (Not a Number) in the feature space and
    # will be excluded below. A try-except clause is used to handle the error.
    try:
        daisy_features = daisy(cell_img, step=2, radius=8)[0]
        fspace2[index,:] = daisy_features[0,:]
    except Exception:
        fspace2[index,:] = np.NaN

# Exclusion of cells that produced an error
# By deleting them from both the feature space and the list of labels,
# the feature space remains mapped onto the segmentation correctly.
# Note that the deletion of samples containing outliers or producing errors
# is a common step in cleaning up data for advanced analysis. However, one
# has to be careful to avoid accidentally introducing bias into the dataset
# through these deletions.
keep = []
for index,label in enumerate(labels):
    if not np.isnan(fspace2[index,0]):
        keep.append(index)
labels2 = labels[keep]
fspace2 = fspace2[keep]

### <font color='orange'>Exercise 3  (Solution)</font>

Just as with the measurements in the main tutorial, it can be interesting to check how the extracted features map onto the actual cells. You can do this with an appropriately colored semi-transparent overlay, just as in the main tutorial.

Have a look at your newly constructed features to see if they make sense. You can also have a look at some of the DAISY features.

Besides the semi-transparent overlays, the population data we now have available can of course also be visualized and investigated by means of scatter plots and boxplots. Feel free to play around a bit!

In [ ]:
### Feature visualizations

# Compute image mapping of 'roundness'
mapped = np.zeros_like(img_green,dtype=np.float)
for row_index,label in enumerate(labels):
    mapped[seg==label] = fspace[row_index,col_labels.index('roundness')]
    
# Transparent overlay
plt.figure(figsize=(7,7))
plt.imshow(img_green,cmap='gray',interpolation='none')
plt.imshow(np.ma.array(mapped,mask=mapped==0),interpolation='none',alpha=0.7) 
plt.show()

# Scatterplot of 'cell area' over 'standard deviation of membrane intensity'
plt.scatter(fspace[:,col_labels.index('cell_area')],fspace[:,col_labels.index('green_asymmetry')])
plt.xlabel("cell area [pixels]")
plt.ylabel("standard deviation of membrane intensity [a.u.]")
plt.show()

# Boxplot of all the manually extracted features
# Note that this looks terrible because of the different scales of the features. 
# This is addressed in the next section!
plt.figure(figsize=(15,5))
plt.boxplot(fspace,labels=col_labels)
plt.show()

## Normalization and Standardization

Many classification and clustering algorithms need features to be normalized and/or standardized, otherwise the absolute value range of the feature could affect the result (for example, you could get a different result if you use cell size in um or in pixels, because the absolute numbers are different).

Normalization in this context generally means scaling your features to a range from 0 to 1. Standardization means centering the features around zero and scaling them to "unit variance" by dividing them by their standard deviation (sometimes called "whitening").

It's worthwhile to read up on normalization/standardization so you can avoid introducing errors/biases. For example, normalization of data with outliers will compress the 'real' data into a very small range. Thus, outliers should
be removed before normalization/standardization.

### <font color='orange'>Exercise 4  (Solution)</font>

In preparation for feature rescaling, find a sensible way to remove outliers from your feature space.

Be mindful of implicit assumptions and potential bias that might come with outlier removal!

In [ ]:
### Removing outliers
# Simple approach: Compute the standard deviation and consider all values that are more than 
# 3 sigma from the mean as outliers. 
# Note: This implicitely assumes a normal distribution of the data, so it may not always be
#       a good idea at all!

# Find outliers
stdevs = np.std(fspace,axis=0)
outliers = np.abs(fspace-np.mean(fspace,axis=0)) > 3 * stdevs

# Remove outliers (whilst preserving mapping of fspace to labels)
keep = np.where(np.sum(outliers,axis=1)==0)[0]
labels = labels[keep]
fspace = fspace[keep]

### <font color='orange'>Exercise 5  (Solution)</font>

Standardize or normalize your feature space as you deem fit, either manually or using a module function.

Don't forget to visualize the result using a boxplot!

In [ ]:
# Center the data around the mean
fspace_c = fspace - np.mean(fspace,axis=0) 

# Whiten
from scipy.cluster.vq import whiten
fspace_w = whiten(fspace_c)

# Same boxplot as above
plt.figure(figsize=(15,5))
plt.boxplot(fspace_w,labels=col_labels)
plt.show()

## Principal Component Analysis (PCA)

The principal components of a feature space are the axes of greatest variance
of the data. By transforming our data to this "relevance-corrected" coordinate
system, we can achieve two things:

1. Usually, most of the variance in a dataset falls onto just a few principal
   components, so we can ignore the other ones as irrelevant, thus reducing
   the number of features whilst maintaining all or most of the information 
   in the data ('dimensionality reduction'). For many downstream analyses,
   this can improve both the quality of the result and the computational performance.
2. Just PCA on its own can yield interesting results. For example, different cell
   populations that are not clearly separated by any single feature may 
   appear separated along a principal component ('clustering'). Furthermore, principal 
   components may correlate with features of the data, which helps us understand the impact
   these features have on downstream analyses. It can also help us understand how abstract
   features such as the DAISY descriptors relate to the image.

### <font color='orange'>Exercise 6  (Solution)</font>

Perform a PCA on your feature space and investigate the results.

You may want to use the PCA implementation of scikit-learn. Algorithms in `sklearn` are provided as "estimator objects". The general workflow for using them is as follows:

1. Instantiate the estimator object and pass it general parameters.
2. Fit the estimator to your data.
3. (Optional: extract information about your data from the estimator.)
4. Transform your data to the reference space of the estimator.
5. (Optional: discard some dimensions of the transformed data, e.g. the dimensions that explain very little of the populations variance in a PCA. This can also be done from the start, by telling the estimator to reduce the data to a set number of dimensions.)

Investigate the results of your PCA using scatter plots (and other visualizations if you like):

- Plot PCs against each other (PC1 vs PC2, ...)
- Plot PCs against features (PC1 vs feature1, feature 2, ...)

In [ ]:
### Performing PCA with scikit-learn

# Instantiate the estimator object
from sklearn.decomposition import PCA
pca_estimator = PCA()

# Fit the ddata
pca_estimator.fit(fspace_w)

# Get useful information: percent of variance explained by each principal component
print "PC-explained variance:", pca_estimator.explained_variance_ratio_

# Transform data to principal component space
fspace_pca = pca_estimator.transform(fspace_w)

In [ ]:
### Visualizing the outcome of the PCA

# Scatterplot of PC1 vs PC2
# NoTEé Uniform data like this yields a rather diffuse cloud,
# whereas more structured data would yield clusters or trajectories.

plt.scatter(fspace_pca[:,0],fspace_pca[:,1])
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.show()

# Scatterplots of PC1 vs selected features
# Note: If a PC correlates with one or more features, we may
# start to get an understanding of the kind of variation that 
# is represented by that PC.

fig,ax = plt.subplots(2,2)

ax[0,0].scatter(fspace_w[:,col_labels.index('cell_area')],fspace_pca[:,0])
ax[0,0].set_title("PC1 vs cell_area")

ax[0,1].scatter(fspace_w[:,col_labels.index('roundness')],fspace_pca[:,0]) 
ax[0,1].set_title("PC1 vs roundness")

ax[1,0].scatter(fspace_w[:,col_labels.index('green_asymmetry')],fspace_pca[:,0]) 
ax[1,0].set_title("PC1 vs green_asymmetry")

ax[1,1].scatter(fspace_w[:,col_labels.index('green_mem_stdev')],fspace_pca[:,0])
ax[1,1].set_title("PC1 vs green_mem_stdev")

plt.tight_layout()
plt.show()

## K-means clustering

If you expect that you can split your population into distinct groups, an easy way of doing so in an unsupervised fashion is k-means clustering. K-means partitions samples into clusters based on their proximity to the cluster's centroid.

### <font color='orange'>Exercise 7 (Solution)</font>

Perform k-means clustering on your data. You can use `sklearn` or `scipy` to do so.

To do so, you have to decide the number of clusters from the start. Just try it with 5 clusters to begin with. You can try clustering raw, normalized/standardized and/or PCA-transformed data to see if there is a difference.

Note that we can't really hope for a clear separation of our population into meaningful clusters because our population of cells is very uniform to begin with.

Visualize your results as color-coded scatterplots and as a semi-transparent map to the image.

In [ ]:
### K-means clustering using scipy

# Perform the clustering
k = 5
import scipy.cluster.vq as svq
k_centroids,k_labels = svq.kmeans2(fspace_pca,k)

# Visualization in PC space
plt.scatter(fspace_pca[:,0],fspace_pca[:,1],c=k_labels,s=30,edgecolor='none') # Color coded data
plt.scatter(k_centroids[:,0],k_centroids[:,1],s=150,c=range(k),marker='*') # Centroids
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.show()

# Semi-transparent overlay on the image: mapping
mapped = np.zeros_like(img_green,dtype=np.float)
for index,label in enumerate(labels):
    mapped[seg==label] = k_labels[index] + 1

# Semi-transparent overlay on the image: plotting
plt.figure(figsize=(7,7))
plt.imshow(img_green,cmap='gray',interpolation='none')
plt.imshow(np.ma.array(mapped,mask=mapped==0),interpolation='none',alpha=0.7) 
plt.show()

### <font color='orange'>Bonus Exercise (Solution)</font>

Can you find/implement a way of objectively choosing the number of clusters for k-means?

<font color='red'>**Warning:**</font> The solution below is probably correct insofar as it uses an Elbow plot, but the computation of the standard deviations is too elaborate and possibly wrong. This will be corrected in a later revision of this tutorial!

In [ ]:
### Choosing the number of clusters using an "Elbow Plot"
# Assuming that we would like to minimize the intra-cluster standard deviation with
# as few clusters as possible, we can plot the resulting stdev for different numbers 
# of clusters and choose the number above which the reduction in the stdev for
# additional clusters is marginal.

# Package the PCA into a function, together with the extraction of the stdev
def pca_stdev(data,k):
    
    # Perform the pca with the current number of clusters
    k_centroids,k_labels = svq.kmeans2(data,k)
    
    # Get the stdev of each cluster's distances from the cluster's centroid
    stdevs = np.zeros(k)
    for i in range(k):
        
        # Find the data points associated to the cluster
        cluster_coords = data[k_labels==i,:]
        
        # Calculate the distance of these datapoints from the centroid
        from scipy.spatial.distance import cdist
        cluster_dists = cdist(cluster_coords,np.expand_dims(k_centroids[i],0))
        
        # Calculate the stdev of these distances
        stdevs[i] = np.std(cluster_dists)
    
    # Average the stdevs for the different clusters
    full_stdev = np.mean(stdevs)
    
    # Return the result
    return full_stdev

# Run for a range of cluster numbers
test_k_list = range(1,13)
k_stdevs = []
np.random.seed(999) # See note 2 below
for test_k in test_k_list:
    k_stdevs.append(pca_stdev(fspace_pca,test_k))

# Plot the elbow plot
plt.plot(test_k_list,k_stdevs)
plt.xlabel("Number of Clusters")
plt.ylabel("Average Cluster STDEV")
plt.show()

# Note 1: The resulting elbow plot is not particularly useful. The ideal outcome would be a plot
#         with a clear 'elbow' (fast decay and then sudden bend into flat curve). In that case,
#         one would choose the point near the 'elbow' as the ideal number of clusters. 
#         This result, however, makes it difficult to choose the number of clusters and probably
#         indicates that this feature space cannot really be partitioned into sensible clusters, 
#         at least not with k-means. This is not unexpected, since we do not see separate 
#         populations when looking at the scatterplots of our features.
#         For a nicer example of an elbow plot, see the result of the PCA in the section on
#         "Graph-Based Analysis" below.

# Note 2: The kmeans implementation used here is initialized with random centroid positions that 
#         are then optimized by the algorithm. Consequently, you will get a slightly different 
#         result each time you run the code. Here, I have ensured reproducibility by adding the
#         line "np.random.seed(999)", which will ensure that the random number generator will 
#         produce the same starting positions each time.
#         Furthermore, to make sure that you get a representative elbow plot, it would make sense 
#         to run the entire algorithm several times with different seeds and then use the average 
#         elbow plot.

## tSNE Analysis

Although PCA is great to reduce and visualize high-dimensional data, it only
works well on linear relationships and global trends. Therefore, alternative 
algorithms optimized for non-linear, local relationships have also been
created.

These algorithms tend to be quite complicated and going into them is beyond 
the scope of this tutorial. This example is intended as a taste of what is out
there and to show people who already know about these methods that they are
available in Python. Note that it can be risky to use these algorithms if 
you do not know what you are doing, so it may make sense to read up and/or to 
consult with an expert before you do this kind of analysis.

This example uses the tSNE implementation in scikit-learn. tSNE (or t-distributed
Stochastic Neighbor Embedding) is a machine learning algorithm that attempts to
project the high-dimensional feature data into just 2 or 3 dimensions, preserving
the proximity between samples as much as possible. In other words, samples that
are close together in the full nD feature space should end up close together in
the reduced 2D/3D feature space.

In [ ]:
### tSNE with scikit-learn

# As with the PCA, the first step is to instantiate an estimator object

# Note 1: Here I enforce a dimensionality reduction to the 2 most important
# components for illustration purposes. The same syntax could be used in the
# skimage implementation of pca to do dimensionality reduction.

# Note 2: random_state works much like numpy.random.seed.

from sklearn.manifold import TSNE
model = TSNE(n_components=2, random_state=999)

# Fitting the data
# Next, we fit thie estimator to our data and transform the data to the new
# frame of reference. Here, this is done in one step directly.
fspace_tsne = model.fit_transform(fspace_w)

# Let's compare the result to the PCA
fig,ax = plt.subplots(1,2,figsize=(7,3))

ax[0].scatter(fspace_pca[:,0],fspace_pca[:,1],c=fspace_w[:,1])
ax[0].set_title("PC1 vs PC2")

ax[1].scatter(fspace_tsne[:,0],fspace_tsne[:,1],c=fspace_w[:,1])
ax[1].set_title("tSNE1 vs tSNE2")

plt.axis('equal')
plt.show()

# Note: Once again, the data used here is too uniform to nicely illustrate
#       how powerful tSNE is at finding structure in multi-dimensional data.
#       The fact that all the points end up in a nice circle (except for
#       some outliers) indicates that tSNE does not find any similarity-based
#       structure in our feature space.

## Graph-Based Analysis

Graphs are a universal way of mathematically describing relationships, be 
they based on similarity, interaction, or virtually anything else. Despite 
their power, graph-based analyses have so far not been used extensively on 
biological imaging data, but as microscopes and analysis algorithms improve,
they become increasingly feasible and will likely become very important in
the future.

The `networkx` module provides various functions for importing and generating
graphs, for operating on and analyzing graphs, and for exporting and visualizing
graphs. The following example shows how a simple graph based on our feature 
space could be constructed and visualized. In doing so, it introduces the `networkx.Graph`
object, which is the core of the `networkx` module.

For this example, I am using the feature space generated by the DAISY descriptor,
so I first need to clean it (remove outliers, standardize) and reduce its dimensionality, much
as we did for the other feature space above.

In [ ]:
### Outlier removal

# Find outliers
# Note: This approach removes many cells, and is thus likely not appropriate
# for the DAISY features. However, I am keeping it for now since it does not
# really matter for this data.
stdevs = np.std(fspace2,axis=0)
outliers = np.abs(fspace2-np.mean(fspace2,axis=0)) > 3 * stdevs

# Remove outliers (whilst preserving mapping of fspace to labels)
keep = np.where(np.sum(outliers,axis=1)==0)[0]
labels2 = labels2[keep]
fspace2 = fspace2[keep]

In [ ]:
### Standardization
fspace2_w = (fspace2 - np.mean(fspace2,axis=0)) / np.std(fspace2,axis=0)

In [ ]:
### Dimensionality reduction with PCA 

# Reduction to 50 components
from sklearn.decomposition import PCA
pca_estimator = PCA(n_components=50,copy=True)
pca_estimator.fit(fspace2_w)    

# Elbow plot of how much variance each PC explains
plt.plot(range(1,51),pca_estimator.explained_variance_ratio_)
plt.xlabel("PC")
plt.ylabel("Explained Variance Ratio")
plt.show()

# Dimensionality reduction to 20 PCs
# The choice of the number 20 is based on the elbow plot!
pca_estimator = PCA(n_components=20,copy=True)
pca_estimator.fit(fspace2_w)  
fspace2_pca = pca_estimator.transform(fspace2_w)

In [ ]:
### Construction of the Graph

# As mentioned above, graphs are a very generic way of representing relations, so there are also
# many ways of generating a graph. Here, I am thresholding the pairwise euclidean distance of the
# cells after dimensionality reduction to 20 features by PCA.
# In other words, I draw an edge between two nodes (cells) in the graph if they are sufficiently
# close together in feature space. This graph thus represents relationships based on "similarity"
# of the cells in feature space.
# Computationally, a graph like this can be represented by an "adjacency matrix", a symmetric array
# of shape (n_cells,n_cells) where 1 denotes an edge and 0 no edge between the cells of the 
# corresponding row and column.

# Calculating pairwise distances
from scipy.spatial.distance import pdist,squareform
graph_pdists = squareform(pdist(fspace2_pca))

# Thresholding pairwise distances
# The threshold is chosen such that the closest 5% of cells will be connected.
# The resulting array is the adjacency matrix of our graph.
graph_thresh = graph_pdists <= np.percentile(graph_pdists,5)

# Cleanup: Since the matrix is symmetric, remove one half of it (and the diagonal)
graph_thresh = np.triu(graph_thresh,k=1)

# Display the resulting adjacency matrix
plt.imshow(graph_thresh,interpolation='none',cmap='gray')
plt.show()

In [ ]:
### From Adjacency Matrix to networkx Graph

# This is pretty simple...
import networkx as nx
G = nx.from_numpy_matrix(graph_thresh)

In [ ]:
### Displaying the graph as a network

# Once we have a networkx Graph object, there are many possibilities for 
# operating on, analyzing, partitioning and visualizing this graph. Here,  
# I limit myself to showing some examples of visualization.

In [ ]:
# Display simple graph
# Get node positions based on graph itself (force-based distribution) and colors from PC1
pos = nx.fruchterman_reingold_layout(G)
plt.figure(figsize=(7,7))
nx.draw(G,pos=pos,cmap=plt.get_cmap('gist_rainbow'),node_color=fspace2_pca[:,0])
plt.show()

In [ ]:
# Display simple graph
# Get positions from PC1 and PC2 and colors from PC1
pos_dict_pca = {}
for index in range(len(labels2)):
    pos_dict_pca[index] = fspace2_pca[index,:2]
plt.figure(figsize=(7,7))
nx.draw(G,pos=pos_dict_pca,cmap=plt.get_cmap('gist_rainbow'),node_color=fspace2_pca[:,0])
plt.show()

In [ ]:
# Display graph nicely overlayed over cells and segmentation

# Use cell centroids as node positions
pos_dict_img = {}
for index,label in enumerate(labels2):
    cen_coords = ndi.measurements.center_of_mass(seg==label)
    pos_dict_img[index] = (cen_coords[1],cen_coords[0])

# Map PC1 onto the segmentation
pca_mapped = np.zeros_like(seg)
for index,label in enumerate(labels2):
    pca_mapped[seg==label] = fspace2_pca[index,0]

# Start plotting
plt.figure(figsize=(12,12))

# Plot image data
plt.imshow(img_green,interpolation='none',cmap='gray')

# Plot semi-transparent segmentation with PC1 coloring
plt.imshow(np.ma.array(pca_mapped,mask=pca_mapped==0),
           interpolation='none',cmap='winter',alpha=0.5)

# Plot nodes on cells with PC1 coloring
nx.draw_networkx_nodes(G,pos=pos_dict_img,
                       cmap='winter',node_color=fspace2_pca[:,0],alpha=0.5,
                       node_size=50,linewidths=0)

# Plot edges
nx.draw_networkx_edges(G,pos=pos_dict_img,
                       edge_color='r',alpha=0.3)

# nx.draw(G,pos=pos_dict_img,
#         cmap=plt.get_cmap('gist_rainbow'),node_color=fspace2_pca[:,0],alpha=0.5,
#         node_size=50,edge_color='0.5')

# Done
plt.axis('off')
plt.tight_layout()
plt.show()

# Note that graph visualization is not an easy task and the use of dedicated 
# software such as Cytoscape may be advisable on occasion. Networkx supports 
# the export of Graphs to various other software.

### <font color='orange'>*Congratulations! You have completed the tutorial!*</font>

<br>

**...but if you now just go back to your work and do nothing, you will forget all you learned within a month or two!**


So, what to do?


- Start applying what you have learned to your own work!


- Stay engaged even if you currently don't need your new skills at work!

    - Play around with data from your work, even if you don't need it at the moment

    - Find yourself an interesting little 'pet project' to play around with

    - Look for tutorials online with additional/advanced content
    
    - Join for seminars/events related to coding and image analysis
        - Check out the [Bio-IT Portal](https://bio-it.embl.de/) for more info! *[internal access only]*
        - Join the [EMBL Coding Club](https://bio-it.embl.de/coding-club/) *[internal access only]*